In [3]:
import pandas as pd
import requests
import bs4
import json
import csv

In [ ]:
def get_data_gov_map():
    '''
    Get all the real esate deals in Tel-Aviv from govmap - from 1998 until now.
    50000000-50006000 include all the range of the street codes in in tel aviv (Note:Part of them empty)
    this function get json data and return datafarme with all the data 
    '''
    url =  "https://ags.govmap.gov.il/Identify/GetDealsByStreetCodes"
    payload = {
      "StreetCodes": [
        50000101
        
      ],
      "Extent": {
        "xmin": 180110.8086,
        "ymin": 666110.696799999,
        "xmax": 186930.7648,
        "ymax": 670915.339299999
      }
    }
    
    data = requests.post(url, json=payload).json()
    df = pd.DataFrame(data["data"])
    
    
    for i in range(50000000,50006000):
        payload = {
          "StreetCodes": [
            int(i)
          ],
          "Extent": {
            "xmin": 185110.8086,
            "ymin": 669110.696799999,
            "xmax": 185730.7648,
            "ymax": 669715.339299999
          }
        }
        try:
            data = requests.post(url, json=payload, timeout=10).json()
        except:
            print(f"Oops! StreetCodes:{i} is'nt working")
 
        
        if data['data'] != []:

            new_df = pd.DataFrame(data["data"])
            df = pd.concat([df,new_df],ignore_index = True)
            print(f'index: {i} , shape:{df.shape}')
            
    return df 
    

In [1]:
url =  "https://www.nadlan.gov.il/Nadlan.REST/Main/GetAssestAndDeals"

payload = {
    "MoreAssestsType": 0,
    "FillterRoomNum": 0,
    "GridDisplayType": 0,
    "ResultLable": "תל אביב -יפו",
    "ResultType": 1,
    "ObjectID": "5000",
    "ObjectIDType": "text",
    "ObjectKey": "UNIQ_ID",
    "DescLayerID": "SETL_MID_POINT",
    "Alert": None,
    "X": 180428.31832654,
    "Y": 665726.5550939,
    "Gush": "",
    "Parcel": "",
    "showLotParcel": False,
    "showLotAddress": False,
    "OriginalSearchString": "תל אביב  יפו",
    "MutipuleResults": False,
    "ResultsOptions": None,
    "CurrentLavel": 2,
    "Navs": [],
    "QueryMapParams": {
        "QueryToRun": None,
        "QueryObjectID": "5000",
        "QueryObjectType": "number",
        "QueryObjectKey": "SETL_CODE",
        "QueryDescLayerID": "KSHTANN_SETL_AREA",
        "SpacialWhereClause": None
    },
    "isHistorical": False,
    "PageNo": 3900,
    "OrderByFilled": "DEALDATETIME",
    "OrderByDescending": True,
    "Distance": 0
}
# 4000 - 2009
# 4200 - 2008
# 4500 - 2007
# 5000 - 2005 
# 5100 - 2004 
response = requests.post(url, json=payload)
json_data = response.json()
df = pd.DataFrame(json_data['AllResults'])


def get_nandlan_data(url, payload,df):
    payload['PageNo'] = 3901
    while payload['PageNo'] < 5200 :   
        print( payload['PageNo'])
        try:

            response = requests.post(url, json=payload)
            json_data = response.json()
            new_df = pd.DataFrame(json_data['AllResults'])
            if new_df.shape[0] != 0 :
                new_rows = pd.DataFrame.from_dict(json_data)
                new_df = pd.DataFrame(json_data['AllResults'])
                df = pd.concat([df,new_df],ignore_index = True)
        except:
            print(f"error:{payload['PageNo']}")
            time.sleep(30)
                    
        
        payload['PageNo'] += 1  
        
    return df
df = get_nandlan_data(url , payload ,df )
df.to_csv('Real_Estate_TLV_Nadlan.csv')



In [2]:
# Save the data in csv file 

# gov_map_df = get_data_gov_map()
# gov_map_df.to_csv('Real_Estate_TLV_GOVMAPS.csv')

# nadlan_df = get_nandlan_data(url,payload,df)
# nadlan_df.to_csv('Real_Estate_TLV_NADLAN.csv')


(151025, 22)